In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import kagglehub

# Download latest version
path = kagglehub.dataset_download("matthieugimbert/french-bakery-daily-sales")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd
# kagglehub, please make this simpler, provide some example, really poor documentation for the api"
fp = path + "/Bakery Sales.csv"
df = pd.read_csv(fp)
cols = df.columns.tolist()
cols.remove("Unnamed: 0")
df = df[cols]
df.head()

In [ ]:
filter_baguette = (df.article == "BAGUETTE")
df["datetime"] = pd.to_datetime(df["date"]  + " " + df["time"])
df_baguette = df[filter_baguette]
cols = ["datetime", "Quantity"]
df_baguette = df_baguette[cols]

In [ ]:
df_daily_baugette = df_baguette.set_index("datetime").resample("D").sum()
df_weekly_baugette = df_baguette.set_index("datetime").resample("W").sum()

In [ ]:
df_weekly_baugette = df_weekly_baugette.reset_index()

In [ ]:
df_weekly_baugette

In [ ]:
zero_counts = (df_daily_baugette.Quantity == 0)
df_daily_baugette[zero_counts].shape


In [ ]:
df_daily_baugette = df_daily_baugette.reset_index()

In [ ]:
df_daily_baugette["Day"] = df_daily_baugette.index
df_daily_baugette["DOW"] = df_daily_baugette.datetime.dt.day_of_week
df_daily_baugette["month"] = df_daily_baugette.datetime.dt.month

In [ ]:
df_daily_baugette["WOY"] = df_daily_baugette.datetime.dt.isocalendar().week


In [ ]:
cols = ["Day","datetime", "DOW", "month", "WOY", "Quantity"]
df_daily_baugette = df_daily_baugette[cols]


In [ ]:
df_daily_baugette.shape

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.figure(figsize=(12, 6))
df_daily_baugette["Quantity"].plot()

## Dependency Check
The purpose of the STL decomposition is to identify underlying components of the series. The purpose of the auto-correlation plots is to get another perspective on the dependence structure. This tells us the variance components that we have to account for if we are going to build a forecasting model. Note that for demand estimation, we are not actually forecasting anything, we need a statistical characterization of the demand **independent** of the sequence order of demand arrival, we only care that the net demand is right, we don't care how it arrives. In forecasting, we do and we have to account for it in the model.

In [ ]:
from statsmodels.tsa.seasonal import STL
daily_baugette_sales = pd.Series(df_daily_baugette["Quantity"].values, index=df_daily_baugette["datetime"])
stl = STL(daily_baugette_sales, period=7)
res = stl.fit()

In [ ]:
decomp_res = {"Trend": res._trend, "Seasonality": res._seasonal, "Noise": res._resid}
df_res = pd.DataFrame.from_dict(decomp_res, orient="columns")
df_res = df_res.reset_index()

In [ ]:
df_res["Day"] = df_res.index + 1

In [ ]:
df_res

## Zero Check
Just want to check if it is a count process dominated by zeros, or if zeros are small in number.

In [ ]:
zero_counts = (df_daily_baugette.Quantity == 0)
df_daily_baugette[zero_counts].shape

In [ ]:
# Using plotly.express
import plotly.express as px
fig = px.line(df_daily_baugette, x='datetime', y="Quantity")
fig.update_layout(
    autosize=False,
    width=1100,
    height=800,
)
fig.show()

In [ ]:
# Using plotly.express
import plotly.express as px
fig = px.line(df_res, x='datetime', y="Trend", title="Trend Cycle Component of Daily Baugette Sales",
             labels = {"Trend": "Number of Baugettes", "datetime": "date"})# Using plotly.express
fig.update_layout(
    autosize=False,
    width=1100,
    height=800,
)
fig.show()

In [ ]:
# Using plotly.express
import plotly.express as px
fig = px.line(df_res, x='datetime', y="Seasonality", title="Seasonality Component of Daily Baugette Sales",
             labels = {"Trend": "Number of Baugettes", "datetime": "date"})# Using plotly.express
fig.update_layout(
    autosize=False,
    width=1100,
    height=800,
)
fig.show()

In [ ]:
# Using plotly.express
import plotly.express as px
fig = px.line(df_res, x='datetime', y="Noise", title="Noise Component of Daily Baugette Sales",
             labels = {"Noise": "Number of Baugettes", "datetime": "date"})# Using plotly.express
fig.update_layout(
    autosize=False,
    width=1100,
    height=800,
)
fig.show()

In [ ]:
df_res["Trend"].plot.kde()
plt.grid(True)

In [ ]:
df_res["Seasonality"].plot.kde()
plt.grid(True)

In [ ]:
import statsmodels.api as sm

In [ ]:
plt.rc("figure", figsize=(12,8))
acf_plot = sm.graphics.tsa.plot_acf(df_daily_baugette["Quantity"] , lags=40)
plt.grid(True)

In [ ]:
plt.rc("figure", figsize=(12,8))
pacf_plot = sm.graphics.tsa.plot_pacf(df_daily_baugette["Quantity"], lags=40, method="ywm")
plt.grid(True)

In [ ]:
df_baguette = df_baguette.reset_index(drop=True)

In [ ]:
df_baguette = df_baguette.rename(columns={"index": "Day"})

In [ ]:
df_baguette["Day"] = df_baguette.index + 1
df_baguette["DOW"] = df_baguette.datetime.dt.day_of_week
df_baguette["month"] = df_baguette.datetime.dt.month

In [ ]:
df_daily_baugette["CWOY"] = df_daily_baugette.apply(lambda x: x["WOY"] + 53 if x.datetime.year == 2022 else x["WOY"], axis=1)

In [ ]:
df_daily_baugette["CWOY"].max()

In [ ]:
df_daily_baugette

In [ ]:
fp = "../data/daily_baugette_sales.csv"
df_daily_baugette.to_csv(fp, index=False)

In [ ]:
df_period_perf  = pd.pivot_table(df_daily_baugette, index= "CWOY", columns= "DOW", values = "Quantity", fill_value=0) 

In [ ]:
HOURS_OPEN = 1
df_period_perf = df_period_perf.apply(lambda x: x.div(HOURS_OPEN), axis=1).round(3).reset_index()

In [ ]:
df_period_perf = df_period_perf.melt(id_vars=["CWOY"], value_vars=[i for i in range(7)], value_name="Quantity")

In [ ]:
df_period_perf

In [ ]:
df_period_perf.Quantity.plot.kde()

In [ ]:
# Using plotly.express
import plotly.express as px
fig = px.scatter(df_period_perf, x='CWOY', y="Quantity")
fig.update_layout(
    autosize=False,
    width=1100,
    height=800,
)
fig.show()